In [ ]:
# imports
import zipfile
from zipfile import ZipFile
import glob
import pandas as pd
import numpy as np
import itertools
import time
import os
import io as sio
import json
import urllib.request

# Check first if photo zipped files are valid!

In [ ]:
currentDir = os.getcwd()
allPhotoZips= []
badfiles= []
allPhotos=[]
for filename in glob.iglob('{}/**/*data-download*.zip'.format(currentDir), recursive=True):
    allPhotoZips.append(filename)
for photoZip in allPhotoZips:
    if zipfile.is_zipfile(photoZip)==False:
        print('Bad Zip file: {0} . Download again!'.format(photoZip))
    else:
        with ZipFile(photoZip) as myzip:
            for info in myzip.infolist():
                if info.file_size==0:
                    badfiles.append(info.filename)
                    #print('Bad archived file: {0} . Need to download manually'.format(info.filename))
                dict={}
                dict["ZipName"]=photoZip
                dict["fileName"]=str(info.filename)
                dict["fileSize"]=int(info.file_size)
                allPhotos.append(dict)
dfRawPhotos=pd.DataFrame(allPhotos)       

In [ ]:
dfRawPhotos.shape

In [ ]:
dfRawPhotos.head()

### Obtain account data files

In [ ]:
currentDir = os.getcwd()
allDataZips= []
for filename in glob.iglob('{}/**/*_part*.zip'.format(currentDir), recursive=True):
    allDataZips.append(filename)

### Build our photo database from albums.json file

In [ ]:
allPhotoAlbums=[]
for dataZip in allDataZips:
    with ZipFile(dataZip) as myzip:
        for jsonFile in [k for k in myzip.namelist() if 'albums' in k]:
            with myzip.open(jsonFile) as jf:
                data=json.load(jf)
                albums = data["albums"]
                for album in albums:
                    for photo in album["photos"]:
                        dict={}
                        dict["albumId"]=int(album["id"])
                        dict["albumName"]=str(album["title"])
                        dict["albumPhotoId"]=int(photo)
                        allPhotoAlbums.append(dict)
dfAlbums = pd.DataFrame(allPhotoAlbums)

In [ ]:
grp = dfAlbums.groupby('albumName').count()
writer = pd.ExcelWriter('albums.xlsx')
grp.to_excel(writer,'albums')
writer.save()

In [ ]:
# Quick way to check if all your photos are in album
dfAlbums.shape

### Build our photo database from photo_nnnnn json files 

In [ ]:
allphotos=[]
for dataZip in allDataZips:
    with ZipFile(dataZip) as myzip:
        for jsonFile in [k for k in myzip.namelist() if 'photo_' in k]:
            with myzip.open(jsonFile) as jf:
                data=json.load(jf)
                dict={}
                dict["id"]=int(data["id"])
                dict["name"]=str(data["name"])
                url=str(data["original"])
                dict["original"]=url
                dict["filename"]=url.rsplit('/', 1)[-1]
                foo=dfAlbums.loc[dfAlbums['albumPhotoId'] == dict["id"]]
                if foo.shape[0]==0:                    
                    dict["albumId"]=int(-1)
                    dict["albumName"]="No Album"
                else:
                    # first album found where this photo belongs
                    dict["albumId"]= foo.iloc[0]['albumId']
                    dict["albumName"]= foo.iloc[0]['albumName']                  
                allphotos.append(dict)

df = pd.DataFrame(allphotos)


In [ ]:
# This should match your total number of items in Flickr
df.shape

### Check if all photos have albums

In [ ]:
df['albumName'].value_counts()

In [ ]:
noalbum=(df.loc[df['albumId'] == -1])

In [ ]:
noalbum.shape

# Creating Folders

Now that we have the albums, lets create folders for them

In [ ]:
for album in df.groupby('albumName').groups:
    directory=os.path.join(os.getcwd(),'Albums',album.strip())
    if not os.path.exists(directory):
        os.makedirs(directory)
# no album
directory=os.path.join(os.getcwd(),'Albums',"No Album")
if not os.path.exists(directory):
    os.makedirs(directory)

# Process the photos and put them where they belong

In [ ]:
for index, row in df.iterrows():
    album=row['albumName']
    directory=os.path.join(os.getcwd(),'Albums',album.strip())
    photoFile=row['filename']
    strId=(str(row['id']))
    albumData=dfRawPhotos[dfRawPhotos['fileName'].str.contains(strId)]
    if albumData.shape[0]==0:
        print('No album for {0}'.format(photoFile))
    else:
        for index2, albumRow in albumData.iterrows():
            if albumRow['fileSize']==0:
                try:
                    urllib.request.urlretrieve(row['original'], os.path.join(directory,albumRow['fileName']))
                except Exception as e:
                    print('Error {} : {}'.format(row['original'],e))
                
            else:
                with ZipFile(albumRow['ZipName']) as myzip:
                    myzip.extract(albumRow['fileName'], directory)
    

In [ ]:
albumData=dfRawPhotos[dfRawPhotos['fileName'].str.contains('8190541672')]
albumData